In [9]:
import numpy as np
import nglview # this import is necessary for rendering of the 3D view
import sys
sys.path.append("../../../matscipy/docs/applications")
from visualisation import show_dislocation

In [10]:
from matscipy.dislocation import BCCScrew111Dislocation
W_params = [3.143, 538.90, 219.93, 178.80]
W_screw = BCCScrew111Dislocation(*W_params, symbol="W")
W_screw_bulk, W_screw_dislo = W_screw.build_cylinder(radius=20)
view1 = show_dislocation(W_screw_dislo, d_name="1/2<111> screw")


disloc SCF 0 |d1-d2|_inf = 0.0014004070350306375
disloc SCF 1 |d1-d2|_inf = 6.702193447782889e-06
disloc SCF 2 |d1-d2|_inf = 3.207553511686001e-08


In [11]:
from matscipy.dislocation import FCCScrew110Dislocation
Ni_params = [3.520, 244.07, 148.85, 125.62]
Ni_screw = FCCScrew110Dislocation(*Ni_params, symbol="Ni")
Ni_screw_bulk, Ni_screw_dislo = Ni_screw.build_cylinder(radius=20, partial_distance=5)
view2 = show_dislocation(Ni_screw_dislo, 
                 partial_distance=5 * Ni_screw.glide_distance,
                 d_name="1/6<112> Shockley partial screw")

disloc SCF 0 |d1-d2|_inf = 0.02578635249849609
disloc SCF 1 |d1-d2|_inf = 0.000968286746150071
disloc SCF 2 |d1-d2|_inf = 0.00019061961503846758
disloc SCF 3 |d1-d2|_inf = 6.2411196626579665e-06
disloc SCF 4 |d1-d2|_inf = 1.1076007152555079e-06
disloc SCF 5 |d1-d2|_inf = 4.976289935676448e-08
disloc SCF 0 |d1-d2|_inf = 0.03226306842287546
disloc SCF 1 |d1-d2|_inf = 0.00214698259185727
disloc SCF 2 |d1-d2|_inf = 0.00026768040270827576
disloc SCF 3 |d1-d2|_inf = 2.7238272580026868e-05
disloc SCF 4 |d1-d2|_inf = 1.5324112329767603e-06
disloc SCF 5 |d1-d2|_inf = 2.938792126871981e-07


In [12]:
from matscipy.dislocation import DiamondGlide60Degree
Si_parms = [5.431, 201.46, 51.64, 118.18]
Si_60_degree_screw = DiamondGlide60Degree(*Si_parms, symbol="Si")
Si_60_degree_screw_bulk, \
Si_60_degree_screw_dislo = Si_60_degree_screw.build_cylinder(radius=20, 
                                                             partial_distance=5)

view3 = show_dislocation(Si_60_degree_screw_dislo, 
                 diamond_structure=True, 
                 scale=0.3, 
                 add_bonds=True, 
                 d_color=[1, 0, 1],
                 partial_distance=5 * Si_60_degree_screw.glide_distance, 
                 d_name=["1/6<112> 90 degree partial screw", 
                         "1/6<112> 30 degree partial screw"])

In [13]:
from ase.build import bulk
from matscipy.dislocation import DiamondGlide90degreePartial

# Data from https://doi.org/10.1080/08927022.2011.602975
alat = 11.2402/2
C11 = 120.31
C12 = 55.87
C44 = 58.26

GaAs = bulk("GaAs", crystalstructure="zincblende", cubic=True, a=alat)


disloc = DiamondGlide90degreePartial(GaAs, C11, C12, C44)

GaAs_bulk, GaAs_dislocation = disloc.build_cylinder(radius=3 * alat)

show_dislocation(GaAs_dislocation, scale=0.35, d_name="GaAs 1/6<112> Beta-90 degree partial", 
                 diamond_structure=True, CNA_color=False, add_bonds=True)

import numpy as np

np.random.seed(1)

species = np.array(GaAs_dislocation.get_chemical_symbols())

Ga_idxs = np.argwhere(species == "Ga")[:, 0]

# Choose random half of the idxs to be In
In_idxs = sorted(np.random.choice(Ga_idxs, size=int(Ga_idxs.shape[0]/2), replace=False))

# Introduce the chemical disorder in In-Ga sites
species[In_idxs] = "In"

InGaAs_bulk = GaAs_bulk.copy()
InGaAs_bulk.set_chemical_symbols(species)

InGaAs_dislocation = GaAs_dislocation.copy()
InGaAs_dislocation.set_chemical_symbols(species)

view4 = show_dislocation(InGaAs_dislocation, scale=0.35, d_name="InGaAs 1/6<112> Beta-90 degree partial", 
                        diamond_structure=True, CNA_color=False, add_bonds=True)

# In and Ga have almost the same default colors in nglview
# so we add another component with the In atoms in red to
# see the chemical disorder better
In_ats = InGaAs_dislocation[In_idxs]
c = view4.add_component(nglview.ASEStructure(In_ats),
                        default_representation=False)
c.add_spacefill(radius=1.0, color="red")
#view.add_spacefill(selection='#Ga', radius=1.0,
#                   color="yellow")
#view

In [14]:
from ipywidgets import HBox

In [15]:
for v in [view1, view4]:
    v.control.zoom(0.15)

for v in [view3, view2]:
    v.control.zoom(0.3)

In [16]:
demo1 = HBox([view1, view2])
demo2 = HBox([view3, view4])

from ipywidgets.embed import embed_minimal_html

embed_minimal_html("W_Ni_demo.html", [demo1])
embed_minimal_html("Si_GaAs_demo.html", [demo2])
